In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from utils.load_csv import load_csv
from utils.load_shp import load_shp

In [2]:
locais_votacao_geoloc = load_csv('locais_votacao_geoloc.csv',
                                 sep=';',
                                 engine='python')

In [3]:
distritos = load_shp('distritos.shp')

In [4]:
locais_votacao_geoloc.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_ORIGEM_VOTO,ST_SECAO_INSTALADA,ST_SECAO_ANULADA,CD_MODELO_URNA,DS_MODELO_URNA,query,end_resultado,lat,lon,score
0,28/10/2024,10:46:35,2024,2,Eleição Ordinária,1,619,Eleições Municipais 2024,06/10/2024,M,...,Urna Eletrônica,Sim,Não,2022,UE 2022,rua alencar araripe 261 são paulo são paulo,"Rua Alencar Araripe, 261, 04253-000, São Paulo",-23.609752,-46.597624,1.0
1,28/10/2024,10:46:35,2024,2,Eleição Ordinária,1,619,Eleições Municipais 2024,06/10/2024,M,...,Urna Eletrônica,Sim,Não,2022,UE 2022,rua alencar araripe 261 são paulo são paulo,"Rua Alencar Araripe, 261, 04253-000, São Paulo",-23.609752,-46.597624,1.0
2,28/10/2024,10:46:35,2024,2,Eleição Ordinária,2,620,Eleições Municipais 2024,27/10/2024,M,...,Urna Eletrônica,Sim,Não,2022,UE 2022,rua alencar araripe 261 são paulo são paulo,"Rua Alencar Araripe, 261, 04253-000, São Paulo",-23.609752,-46.597624,1.0
3,28/10/2024,10:46:35,2024,2,Eleição Ordinária,2,620,Eleições Municipais 2024,27/10/2024,M,...,Urna Eletrônica,Sim,Não,2022,UE 2022,rua alencar araripe 261 são paulo são paulo,"Rua Alencar Araripe, 261, 04253-000, São Paulo",-23.609752,-46.597624,1.0
4,28/10/2024,10:46:35,2024,2,Eleição Ordinária,1,619,Eleições Municipais 2024,06/10/2024,M,...,Urna Eletrônica,Sim,Não,2022,UE 2022,rua alencar araripe 261 são paulo são paulo,"Rua Alencar Araripe, 261, 04253-000, São Paulo",-23.609752,-46.597624,1.0


In [5]:
distritos.head()

,cd_identif,cd_identi0,cd_distrit,nm_distrit,sg_distrit,tx_escala,sg_fonte_o,dt_criacao,dt_atualiz,cd_usuario,cd_tipo_di,qt_area_qu,qt_area_me,geometry
0,8583465.0,31.0,25,CIDADE TIRADENTES,CTI,1:5000,GEOGSG,None,None,None,199299399,14.939,14939046.96,"POLYGON ((357428.688 7387653.528, 357433.497 7..."
1,8583484.0,18.0,46,JARDIM SAO LUIS,JDS,1:5000,GEOGSG,None,None,None,199299399,26.048,26047652.36,"POLYGON ((321618.419 7377778.152, 321559.629 7..."
2,8583488.0,9.0,49,LIBERDADE,LIB,1:5000,GEOGSG,None,None,None,199299399,3.651,3651129.57,"POLYGON ((332504.515 7392094.984, 332505.952 7..."
3,8583453.0,4.0,13,CACHOEIRINHA,CAC,1:5000,GEOGSG,None,None,None,199299399,13.572,13571736.73,"POLYGON ((330933.585 7402181.544, 330910.867 7..."
4,8583416.0,29.0,72,SAO LUCAS,SLU,1:5000,GEOGSG,None,None,None,199299399,9.674,9674427.42,"POLYGON ((340534.808 7391261.575, 340552.031 7..."


In [6]:
locais = locais_votacao_geoloc.copy()
locais['geometry'] = locais.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
locais_gdf = gpd.GeoDataFrame(locais, geometry='geometry', crs="EPSG:4326")

distritos_gdf = distritos.to_crs("EPSG:4326")

joined = gpd.sjoin(locais_gdf, distritos_gdf[['nm_distrit', 'geometry']], 
                   how='left', 
                   predicate='within')

locais_votacao_geoloc['distrito'] = joined['nm_distrit']

In [14]:
locais_votacao_geoloc[['DS_LOCAL_VOTACAO_ENDERECO', 'query', 'end_resultado', 'score', 'lat', 'lon', 'distrito']].sample(5)

,DS_LOCAL_VOTACAO_ENDERECO,query,end_resultado,score,lat,lon,distrito
25800,"AVENIDA CELSO DOS SANTOS, 375",avenida celso dos santos 375 são paulo são paulo,"Avenida Celso dos Santos, 375, 04658-240, São ...",1.0,-23.667420,-46.669131,CIDADE ADEMAR
13285,"RUA CRISTÓVÃO DE VASCONCELOS, 86A",rua cristóvão de vasconcelos 86 são paulo são ...,"Rua Cristóvão de Vasconcelos, 86, 03978-000, S...",1.0,-23.615694,-46.500433,SAPOPEMBA
72402,"RUA LUIZ VIANA, 110",rua luiz viana 110 são paulo são paulo,"Rua Luiz Viana Andrade, 110, 14500-000, Ituverava",1.0,-20.249208,-47.873545,NaN
73088,"RUA PANGAUÁ, 210",rua pangauá 210 são paulo são paulo,"Rua Pangauá, 210, 03665-010, São Paulo",1.0,-23.529990,-46.498625,PENHA
13969,"RUA MINISTRO MÁRIO DAVID ANDREAZZA, 94",rua ministro mário david andreazza 94 são paul...,"Rua Ministro Mário David Andreazza, 94, 04849-...",1.0,-23.742789,-46.671020,GRAJAU


In [15]:
missing = locais_votacao_geoloc[locais_votacao_geoloc['distrito'].isna()]
missing.shape

(1971, 43)